# Overview
We constructed a linear regression model through an iterative process to determine how house characteristics correlated with sale price. The model was trained and tested on a dataset containing 21,597 sales of houses in King County, Washington from May 2014 to May 2015. This model can be used by real estate companies, such as Every Door Real Estate, to determine appropriate housing prices from a house's features. Setting the right price for a house can improve sales and comissions for that company. We used recursive feature elimination with cross-validation to decide which features to include in our model. Our final model had an $R^{2}$ of {}, explaining of the variance in price. The RMSE, the average error of our model in units of price, was $92,528. Two of our key measures were -measure1- and -measure2-. All else held equal, a house that has been renovated has a mean price of \\$9000 greater than houses that were not renovated.

# Business Problem

In the housing market, home sellers and realtors have different goals. A seller's ultimate goal is to sell the house at the highest price. A real estate agent's goal is to sell a higher volume of properties in order to maximize commissions. There is a fine balance between these two goals: the agent aims to sell the property as quickly as possible, but at a high enough value to satisfy their client. A realistic understanding of home prices is a necessity to optimize this tradeoff.

Our client is Every Door Real Estate, a real estate company in Seattle, Washington. Every Door Real Estate represents individuals selling their moderately price homes. This project aims at developing a linear regression model to help Every Door understand how different home features correlate with home prices. The model is constructed using historical home sales in King County, Washington. Every Door can use this model to determine an appropriate range of values for a home based upon its characteristics. This information can be used to determine how much a home is valued on the market and whether a home seller is over- or under-valuing their property.

# Data Understanding
For this analysis, we will utilize the "King County Housing Price from May 2014- May 2015" created by the Center for Spatial Data Science (https://spatial.uchicago.edu). It contains 21,597 entries with 21 different columns about home sales in the Seattle metropolitan area. We've included some descriptive statistics. This dataset provides us with information suitable for linear regression analysis to predict home prices in the Seattle metroplitan area. This robust dataset contains information about previous home selling prices along with multiple features of the home at time of sale. One limitation of this dataset - is that it represents a single year of data, another limitation is that it isnt current data. Ideally to make relevant and accurate predictions we'd need data that is current. Even if this data is not particularly useful at captuing current market trends, it is still useful at helping us evaluate if linear regression is a good fit to model these relationships.

In [1]:
#import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stats
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
from sklearn.dummy import DummyRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
#from random import gauss
#from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [2]:
#ignore pairplot and graph warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
housing = pd.read_csv('data/kc_house_data.csv')
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
housing.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
housing.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

## Observations: 

- We have several columns with categorical values:
    1. waterfront      
    2. view             
    3. condition       
    4. grade           
    5. zipcode
    
    

- Also the column sqft_basement is an object. Using our domain knowledge - we feel that it should be numerical.
  We'll take a look at this in the next section.

- We have several null values in the waterfont and yr_renovated columns. Yr_renovated seems self-explanatory - a         null value means that the house was not renovated. 
- We have a small number of null values in the view column.  
- In the next section we'll explore these columns - waterfront, view, sqft_basement, and yr_renovated columns
  along with a few other columns to try and prepare our data for further analysis.


# Data Preparation 
In this section we'll examine several of our columns and try to clean them up in order to analyse them further.
We'll figure out how to handle null values in several of our columns, ensure the minimum and maximum values are what we expect for columns that contain a measurement of time, and investigate our 'id' column to take a look at instances
where the id is duplicated. We'll also ensure the columns are in appropriate data types.


In [7]:
#examining response values in the waterfront column
print(housing['waterfront'].isna().sum())
housing['waterfront'].value_counts()

2376


NO     19075
YES      146
Name: waterfront, dtype: int64

In [8]:
#taking a look at instances where we have a null value in waterfront and a "NONE" value in the view category
housing[(housing['waterfront'].isna()) & (housing['view'] == "NONE")]


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
10,1736800520,4/3/2015,662500.0,3,2.50,3560,9796,1.0,NaN,NONE,...,8 Good,1860,1700.0,1965,0.0,98007,47.6007,-122.145,2210,8925
23,8091400200,5/16/2014,252700.0,2,1.50,1070,9643,1.0,NaN,NONE,...,7 Average,1070,0.0,1985,NaN,98030,47.3533,-122.166,1220,8386
40,5547700270,7/15/2014,625000.0,4,2.50,2570,5520,2.0,NaN,NONE,...,9 Better,2570,0.0,2000,NaN,98074,47.6145,-122.027,2470,5669
55,9822700295,5/12/2014,885000.0,4,2.50,2830,5000,2.0,NaN,NONE,...,9 Better,2830,0.0,1995,0.0,98105,47.6597,-122.290,1950,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,2025049203,6/10/2014,399950.0,2,1.00,710,1157,2.0,NaN,NONE,...,7 Average,710,0.0,1943,0.0,98102,47.6413,-122.329,1370,1173
21578,5087900040,10/17/2014,350000.0,4,2.75,2500,5995,2.0,NaN,NONE,...,8 Good,2500,0.0,2008,0.0,98042,47.3749,-122.107,2530,5988
21586,844000965,6/26/2014,224000.0,3,1.75,1500,11968,1.0,NaN,NONE,...,6 Low Average,1500,0.0,2014,0.0,98010,47.3095,-122.002,1320,11303
21587,7852140040,8/25/2014,507250.0,3,2.50,2270,5536,2.0,NaN,NONE,...,8 Good,2270,0.0,2003,0.0,98065,47.5389,-121.881,2270,5731


In [9]:
#filling in 'NO' for null values in our waterfront column
housing['waterfront'].fillna(value='NO', inplace=True)

In [10]:
#454 columns with '?' for a value. That explains why the data type came up as an object
housing['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
1135.0        1
768.0         1
875.0         1
207.0         1
1880.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [11]:
# create new column sqrt_basement2 which is sqrt_living - sqrt-above. Addresses sqft_basement ? values.
housing['sqft_basement2'] = housing['sqft_living'] - housing['sqft_above']
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,400
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,910
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,0


In [12]:
# has 3842 nan values, also has some 0.0 values
print(housing['yr_renovated'].isna().sum())
housing['yr_renovated'].value_counts()

3842


0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [13]:
#creating new column "was_renovated". Assuming Nan value and 0.0 - means home was not renovated.
housing['yr_renovated'].fillna(0, inplace=True)
housing['was_renovated'] = housing['yr_renovated'] != 0.0

In [14]:
#replacing 0.0 in yr_renovated column with associated value in yr_built
housing.loc[housing['yr_renovated'] == 0, ['yr_renovated']] = housing['yr_built']

In [15]:
#sanity check
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2,was_renovated
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,0.0,1955,1955.0,98178,47.5112,-122.257,1340,5650,0,False
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,400,True
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,0.0,1933,1933.0,98028,47.7379,-122.233,2720,8062,0,False
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,910.0,1965,1965.0,98136,47.5208,-122.393,1360,5000,910,False
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,0.0,1987,1987.0,98074,47.6168,-122.045,1800,7503,0,False


In [16]:
#taking a look at the 63 null values in the view column
print(housing['view'].isna().sum())
housing['view'].value_counts()

63


NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [17]:
#assuming a value of "NONE" for nulls in the view column
housing['view'].fillna(value='NONE', inplace=True)

In [18]:
#investigating outlier bedrooms
housing[housing['bedrooms'] > 8]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2,was_renovated
4092,1997200215,5/7/2014,599999.0,9,4.50,3830,6988,2.5,NO,NONE,...,1380.0,1938,1938.0,98103,47.6927,-122.338,1460,6291,1380,False
4231,2902200015,1/6/2015,700000.0,9,3.00,3680,4400,2.0,NO,NONE,...,850.0,1908,1908.0,98102,47.6374,-122.324,1960,2450,850,False
6073,9822700190,8/8/2014,1280000.0,9,4.50,3650,5000,2.0,NO,NONE,...,1120.0,1915,2010.0,98105,47.6604,-122.289,2510,5000,1120,True
8537,424049043,8/11/2014,450000.0,9,7.50,4050,6504,2.0,NO,NONE,...,0.0,1996,1996.0,98144,47.5923,-122.301,1448,3866,0,False
8748,1773100755,8/21/2014,520000.0,11,3.00,3000,4960,2.0,NO,NONE,...,600.0,1918,1999.0,98106,47.5560,-122.363,1420,4960,600,True
13301,627300145,8/14/2014,1150000.0,10,5.25,4590,10920,1.0,NO,AVERAGE,...,2090.0,2008,2008.0,98004,47.5861,-122.113,2730,10400,2090,False
15147,5566100170,10/29/2014,650000.0,10,2.00,3610,11914,2.0,NO,NONE,...,600.0,1958,1958.0,98006,47.5705,-122.175,2040,11914,600,False
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,NO,NONE,...,580.0,1947,1947.0,98103,47.6878,-122.331,1330,4700,580,False
16830,8823900290,3/17/2015,1400000.0,9,4.00,4620,5508,2.5,NO,NONE,...,750.0,1915,1915.0,98105,47.6684,-122.309,2710,4320,750,False
18428,8823901445,3/13/2015,934000.0,9,3.00,2820,4480,2.0,NO,NONE,...,940.0,1918,1918.0,98105,47.6654,-122.307,2460,4400,940,False


In [19]:
#replacing 33 bedrooms with 3. we feel confident in assuming this was a typo based on the bathrooms and sqft_living
housing['bedrooms'] = housing['bedrooms'].replace(33, 3)

In [20]:
housing['id'].value_counts()

795000620     3
1825069031    2
2019200220    2
7129304540    2
1781500435    2
             ..
7812801125    1
4364700875    1
3021059276    1
880000205     1
1777500160    1
Name: id, Length: 21420, dtype: int64

In [21]:
#checking instance where id is duplicated.
idcheck1 = housing[housing['id'] == 795000620]
idcheck1

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2,was_renovated
17588,795000620,9/24/2014,115000.0,3,1.0,1080,6250,1.0,NO,NONE,...,0.0,1950,1950.0,98168,47.5045,-122.33,1070,6250,0,False
17589,795000620,12/15/2014,124000.0,3,1.0,1080,6250,1.0,NO,NONE,...,0.0,1950,1950.0,98168,47.5045,-122.33,1070,6250,0,False
17590,795000620,3/11/2015,157000.0,3,1.0,1080,6250,1.0,NO,NONE,...,0.0,1950,1950.0,98168,47.5045,-122.33,1070,6250,0,False


In [22]:
#taking a look at another instance where id is duplicated.
idcheck2 = housing[housing['id'] == 1825069031]
idcheck2

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2,was_renovated
3946,1825069031,8/14/2014,550000.0,4,1.75,2410,8447,2.0,NO,GOOD,...,350.0,1936,1980.0,98074,47.6499,-122.088,2520,14789,350,True
3947,1825069031,10/16/2014,550000.0,4,1.75,2410,8447,2.0,NO,GOOD,...,350.0,1936,1980.0,98074,47.6499,-122.088,2520,14789,350,True


In [23]:
#sanity check
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement2,was_renovated
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,0.0,1955,1955.0,98178,47.5112,-122.257,1340,5650,0,False
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,400,True
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,0.0,1933,1933.0,98028,47.7379,-122.233,2720,8062,0,False
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,910.0,1965,1965.0,98136,47.5208,-122.393,1360,5000,910,False
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,0.0,1987,1987.0,98074,47.6168,-122.045,1800,7503,0,False


## Summary
After examining and cleaning our data set we've decided upon the following:


**Waterfront view**

- the only options are yes or no. 
   1. Any property on a waterfront will have a value other than NONE in the view column. Since there are 2110 rows that have a NaN value in waterfront and a none value in the view column in our data, it seems safer to assume NO as the default for waterfront.
   
   
   
**View**

- several options - none, nan, good, excellent, average, and fair.
    1. Since the porportion of "none" was the average response we feel safe in replacing the small percent (63/21597) of nan values in the view category with the average response of none.



**yr_renovated**
- we created a new column "was_renovated". We gave it Boolean values. 
- We made the assumption that a Nan value and 0.0 - meant the home was not renovated and gave a value of false
    Similarly for homes that has any other sate in the yr_renovated column were assigned a response of true in
    the new "was_renovated" column. 
- We also replaced the 0.0 in yr_renovated column with associated value in yr_built



**sqft_basement**
- We created a new column sqrt_basement2 which is sqrt_living - sqrt-above. This addressed the instances where there   ? values in the sqft_basement column.



**bedrooms**
- We replaced 33 bedrooms with 3. We felt confident in assuming this was a typo based on the bathrooms and sqft_living



**id**
- Keep instances where id's are duplicated - each row represented a different sale as seens by the different sale       dates.


In [24]:
##exporting cleaned data frame
housing.to_csv('./data/kc_house_data_cleaned.csv')

In [25]:
#Exported data to QGIS to obtain distance from water column

## Feature Engineering  
Let's further explore some of our variables. In this section we created some new feature columns to try and eliminate multicollinearity with several of our features. We  We've also imported a new data set containing a column called "distance to water". This column is a numeric calculation of the distance of each house using latitude to longitude to the closest body of water.

In [26]:
![housing_location](./images/housing_location.png)

zsh:1: unknown file attribute: i


In [27]:
![housing_location](./images/housing_location.png)

zsh:1: unknown file attribute: i


In [28]:
##using QGIS import create distance from waterbody column.
cleaned_housing = pd.read_csv('data/kc_water_dist_homes.csv')
cleaned_housing.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/kc_water_dist_homes.csv'

In [ ]:
#checking for feature independence
mask = np.triu(np.ones_like(cleaned_housing.corr(), dtype=bool))

plt.figure(figsize=(20,10))

cor = cleaned_housing.corr().abs()
sns.heatmap(cor, mask=mask, annot=True);

In [ ]:
#correlation based on our 'price' target variable
cleaned_housing.corr().price.sort_values(ascending=False)

In [ ]:
## creating new column ratios to adjust for multicollinearity between variables
cleaned_housing['bed_bath_ratio'] = (cleaned_housing['bedrooms'] /  cleaned_housing['bathrooms'])
cleaned_housing['bath_bed_ratio'] = (cleaned_housing['bathrooms'] /  cleaned_housing['bedrooms'])
cleaned_housing['sqft_living_to_bedroom_ratio'] = (cleaned_housing['sqft_living'] /  cleaned_housing['bedrooms'])
cleaned_housing['sqft_living_to_bathroom_ratio'] = (cleaned_housing['sqft_living'] /  cleaned_housing['bathrooms'])
cleaned_housing['ratio_sqft_lot_living'] = (cleaned_housing['sqft_lot'] /  cleaned_housing['sqft_living'])
cleaned_housing['ratio_sqft_living_lot'] = (cleaned_housing['sqft_living'] /  cleaned_housing['sqft_lot'])

In [ ]:
#creation of two new columns "age at sale" and "age renovation at sale"
cleaned_housing['age_at_sale'] = (cleaned_housing['date'].str[:4].astype(int) - cleaned_housing['yr_built']).astype(int)
cleaned_housing['age_renovation_at_sale'] = (cleaned_housing['date'].str[:4].astype(int) - cleaned_housing['yr_renovated']).astype(int)

In [ ]:
#cleaning up column names, changing column to string and dropping columns
cleaned_housing.rename(columns = {'Hub distance_HubDist':'Distance_to_Water'}, inplace = True)
cleaned_housing['zipcode'] = cleaned_housing['zipcode'].astype(str)
cleaned_housing = cleaned_housing.drop(['date', 'field_1', 'sqft_basement'], axis=1)

# Target Exploration
Exploring the distribution of our target variable.

In [ ]:
min = cleaned_housing['price'].min()
max = cleaned_housing['price'].max()
mean = cleaned_housing['price'].mean()

print (f"The sale price range of homes sold is {min} to {max}")
print (f"The mean sale price of homes was {mean}")

In [ ]:
label = cleaned_housing['price']
fig, ax = plt.subplots(2, 1, figsize = (9,12))

# Plot the histogram   
ax[0].hist(label, bins=100)
ax[0].set_ylabel('Frequency')
ax[0].axvline(label.mean(), color='magenta', linestyle='dashed', linewidth=4)
ax[0].axvline(label.median(), color='cyan', linestyle='dashed', linewidth=4)
ax[0].axvline(label.max(), color='red', linestyle='dashed', linewidth=4)

# Plot the boxplot   
ax[1].boxplot(label, vert=False)
ax[1].set_xlabel('price')
fig.suptitle('Price Distribution');

In [ ]:
print ('Skewness =', stats.skew(cleaned_housing['price']))
print ('Kurtosis =', stats.kurtosis(cleaned_housing['price']))

## Observations

- price is normally distributed but has a significant right tail skew.
- Since the peak of the distribution is to the left of our mean, price is positively skewed.    
- This means that more than half of the houses in our dataset sold for less than the average price $540,000. 
- Our kurtosis and skew are high - we expect to see a positive skew and tail.
- Looking at our box plot - this illustrates that clearly - we have a number of outliers that sold for 
  significantly more than our average.
- Moving forward, we will need to account for some of this skewdness in our models.

# Train Test Split 

In [ ]:
#Setting up train test split
X = cleaned_housing.drop(['price'], axis=1)
y = cleaned_housing['price']

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=57)

In [ ]:
#Combining splits to get test and train complete df's for instances when we will want to use the entire dataset
train_df = pd.concat([y_train, X_train], axis=1)
test_df = pd.concat([y_test, X_test], axis=1)

In [ ]:
# 15117 entries
X_train.info()

In [ ]:
# 6480 entries
X_test.info()

In [ ]:
y_train.mean()

## Dummy Regressor
This dummy regressor is the simplest approach to the problem. In our case the dummy regressor simply predicts the mean of the training set. Simply put, for each new home price we want to predict - the dummy regressor predicts that price as the average price of homes in our training data ($540,000). 

In [ ]:
#DummyRegressor scores for train and test sets
dummy_lr = DummyRegressor()
dummy_lr.fit(X_train, y_train)

print("Dummy Regressor Model:")

print(dummy_lr.score(X_train, y_train))
print(dummy_lr.score(X_test, y_test))

# Baseline Model (Simple Regression Models)

**Simple Model Regression**

 We have one simple model and based upon what we see we'll build some iterations upon it.

### Simple Regression Model

In [ ]:
#our most correlated feature is sqft_living. Let's take a look at this visually.
sns.set_style('whitegrid')

sns.lmplot(x ='sqft_living', y ='price',ci=None, data = train_df).set(title='Living Space Square Footage vs. Selling Price')
plt.xlabel("Living Space (sqft)", fontsize = 13)
plt.ylabel("Selling Price", fontsize = 13);



In [ ]:
#using subset of train df

simple_df = train_df.filter(['price', 'sqft_living'], axis=1)
simple_model = sm.formula.ols('price ~ sqft_living',simple_df).fit()
simple_model_summary = simple_model.summary()
simple_model_summary

In [ ]:
p = sns.distplot(simple_model.resid);
plt.xlabel("Residual Error", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
p.set_title("Distribution of Residual Error");

resid = simple_model.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].hist(resid)
ax[0].set_xlabel("Residual Error")
ax[0].set_ylabel("Count")
ax[0].set_title(label="Histogram of Residual Error");

ax[1].scatter(x=simple_model.fittedvalues,y=resid)
ax[1].set_xlabel("Predicted Values")
ax[1].set_ylabel("Residual Error")
ax[1].set_title(label="Test for Homoscedasticity")

plt.style.use('ggplot')
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True);

In [ ]:
print(f'R-squared training model is {round(simple_model.rsquared,2)}')
print(f'P-Value training model is {simple_model.f_pvalue}')
print(f'Price standard error training model is {round(simple_model.bse,0)[0]}')
print(f'sqft_living coef training model is {round(simple_model.params[1])}')


### Simple Regression Model - refining 1

- our residual error shows a significant right tail skew
- also our test for Homoscedasticity failed.
- lets try and remove our heavy outliers to see if that improves our model assumptions

In [ ]:
#Model 1 - Simple Regression 2
#using subset of train_df. 
#refining model by dropping price values that are +3std away from the mean.

simple_std = simple_df['price'].std()
simple_mean = simple_df['price'].mean()
simple_drop = (simple_std *3) + simple_mean
simple_drop

In [ ]:
#dropping rows where homes sold for more than 3 std away from the mean
simple_df.drop(simple_df[ simple_df['price'] >= 1652910 ].index, inplace = True)

In [ ]:
#14838, current rows. dropped 279.
simple_df.info()

In [ ]:
simple_model2 = sm.formula.ols('price ~ sqft_living',simple_df).fit()
simple_model2_summary = simple_model2.summary()
simple_model2_summary

In [ ]:
print(f'R-squared training model is {round(simple_model.rsquared,2)}')
print(f'P-Value training model is {simple_model.f_pvalue}')
print(f'Price standard error training model is {round(simple_model.bse,0)[0]}')
print(f'sqft_living coef training model is {round(simple_model.params[1])}')
print()
print(f'R-squared simple_model2 is {round(simple_model2.rsquared,2)}')
print(f'P-Value simple_model2 is {simple_model2.f_pvalue}')
print(f'Price standard error simple_model2 is {round(simple_model2.bse,0)[0]}')
print(f'sqft_living coef simple_model2 is {round(simple_model2.params[1])}')

In [ ]:
p = sns.distplot(simple_model2.resid);
plt.xlabel("Residual Error", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
p.set_title("Distribution of Residual Error");

resid = simple_model2.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].hist(resid)
ax[0].set_xlabel("Residual Error")
ax[0].set_ylabel("Count")
ax[0].set_title(label="Histogram of Residual Error");

ax[1].scatter(x=simple_model2.fittedvalues,y=resid)
ax[1].set_xlabel("Predicted Values")
ax[1].set_ylabel("Residual Error")
ax[1].set_title(label="Test for Homoscedasticity")

plt.style.use('ggplot')
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True);

### Simple Regression Model - refining 2
- our residual error is improved
- also our test for Homoscedasticity looks better but our QQ plot still shows that heavy right tail skew
- lets try and remove take the log of our target variable to improve this

In [ ]:
#taking the log of our target variable price
logprice = np.log(simple_df['price'])
simple_model3 = sm.formula.ols('logprice ~ sqft_living',simple_df).fit()
simple_model3_summary = simple_model3.summary()
simple_model3_summary


In [ ]:
print(f'R-squared training model is {round(simple_model.rsquared,2)}')
print(f'P-Value training model is {simple_model.f_pvalue}')
print(f'Price standard error training model is {round(simple_model.bse,0)[0]}')
print(f'sqft_living coef training model is {round(simple_model.params[1])}')
print()
print(f'R-squared simple_model2 is {round(simple_model2.rsquared,2)}')
print(f'P-Value simple_model2 is {simple_model2.f_pvalue}')
print(f'Price standard error simple_model2 is {round(simple_model2.bse,0)[0]}')
print(f'sqft_living coef simple_model2 is {round(simple_model2.params[1])}')
print()
print(f'R-squared simple_model3 is {round(simple_model3.rsquared,2)}')
print(f'P-Value simple_model3 is {simple_model3.f_pvalue}')
print(f'Price standard error simple_model3 is {round(simple_model3.bse,0)[0]}')
print(f'sqft_living coef simple_model3 is {round(simple_model3.params[1])}')

In [ ]:
p = sns.distplot(simple_model3.resid);
plt.xlabel("Residual Error", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
p.set_title("Distribution of Residual Error");

resid = simple_model3.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].hist(resid)
ax[0].set_xlabel("Residual Error")
ax[0].set_ylabel("Count")
ax[0].set_title(label="Histogram of Residual Error");

ax[1].scatter(x=simple_model3.fittedvalues,y=resid)
ax[1].set_xlabel("Predicted Values")
ax[1].set_ylabel("Residual Error")
ax[1].set_title(label="Test for Homoscedasticity")

plt.style.use('ggplot')
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True);

## Observations
- Our intial model gives the following formula : y = 284 * ('sqft_living') - 50,650.
- Sqft_living accounts for about 50% on average of the variance in our sale price.
- Each unit increase of Sqft_living increases the selling price of homes by on average about $284.
- We noticed that our model was not meeting the Homoskedasticity or normal error distribution assumptions so we tried   a few iterations to account for this.
- Model assumptions:
    1. linearity between feature and target variable - illustrated by positive correlation of 0.702
    2. low multicolinearity between target and feature variable - the lowest condition number we achieved was 5,630 - 
        so this assumption wasnt met.
    3. normally distributed errors with a mean of 0. 
       the QQ plot and histogram from model 1 show a significant right tail skew. The plots from iterations 2 and 3
       show a normal distribution. 
    4. Homoskedasticity - variance is constant. Assumption was met in our model iterations 2 and 3.
    5. Errors are not correlated.
    
- Our R2 actually decreased with each successive iteration. When comparing this base model to our multiple regression 
  models we'll use the average R2 of 0.45.   
- We also decided to keep these changes because our model assumptions are met in the last iteration.

# Multiple Regression Models

For our first multiple regression model, we'll create a model with all our numeric columns.

In [ ]:
#recalling our X_train and y_train df's from our previous split
X_train.info()

In [ ]:
#creating new numeric df from our train split
X_train_numeric =X_train.filter(['bedrooms','bathrooms','sqft_living','sqft_lot','floors',
                                    'sqft_above','yr_built','yr_renovated','lat','long','sqft_living15',
                                    'sqft_lot15','sqft_basement2','Distance_to_Water' ], axis=1)
X_train_numeric.info()


In [ ]:
#evaluating numeric features
scatterplot_data = X_train_numeric.drop("sqft_living", axis=1) #already graphed above

fig, axes = plt.subplots(ncols=3, nrows=5, figsize=(12, 8))
fig.set_tight_layout(True)

for index, col in enumerate(scatterplot_data.columns):
    ax = axes[index//3][index%3]
    ax.scatter(X_train_numeric[col], y_train, alpha=0.2)
    ax.set_xlabel(col)
    ax.set_ylabel("price")

    
fig.delaxes(axes[4][1])
fig.delaxes(axes[4][2])

### Muliple Regression Model 1
- Model created with all our numeric columns from our original X_train dataset.

In [ ]:
#model with all our numeric features
multi_model = LinearRegression()
##performing 3 separate train/tets splits on our X_train, y_train
splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
#performing regression on all columns in our dataset
multi_model_scores = cross_validate(
    estimator=multi_model,
    X=X_train_numeric,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Current Model")
print("Train score:     ", multi_model_scores["train_score"].mean())
print("Validation score:", multi_model_scores["test_score"].mean())



**Observations**
- with an r2 of 0.61 ( validation score) is higher than the 0.45 mean r2 from our 3 simple models iterations, 
- we need to see more information about our multicollinearity, pvalues, and coeficients. Lets run the regression 
  with the statsmodel method to ensure we can check the condition number


In [ ]:
sm.OLS(y_train, sm.add_constant(X_train_numeric)).fit().summary()

**Observations**

- several things are of note - our condition number is really high indicating an issue with multicollinearity
  which is expected - considering we threw everything at this model. We know that there is some multicollinearity
  between some of our sqft variable.
- Interestingly, all our variables are statistically significant at an alpha=.05 level.
- our coefficients are all very different, which makes interpretating this model difficult.
- Lets try removing some variables and scaling our data so we can have our coefficients in the same scale.
- To do this we'll use the recursive feature module from sklearn

In [ ]:
#using RFE modedule to select for the top 4 features
#scaling our X_train_numeric dataset
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select =4)

ss = StandardScaler()
ss.fit(X_train_numeric)
X_train_numeric_scaled = ss.transform(X_train_numeric)
select.fit(X= X_train_numeric_scaled, y=y_train)
select.support_

**Observations**
- With Recursive Feature Elimination - we're selecting the top 4 features to include in our model
   these are:
   1. sqft_living
   2. sqft_above 
   3. latitude  
   4. sqft_basement
- we've also scaled our features so our coefficients should be more interpretable. 
- Lets run another regression model with these four features included.

### Muliple Regression Model 1b

In [ ]:
#creating new df with only the relevant 4 columns
X_train_numeric2 = X_train_numeric.filter(['sqft_living','sqft_above','lat','sqft_basement2'], axis=1)

multi_model_b = LinearRegression()
##performing 3 separate train/tets splits on our X_train, y_train
splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
#performing regression on all columns in our dataset
multi_model_b_scores = cross_validate(
    estimator=multi_model_b,
    X=X_train_numeric2,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Current Model")
print("Train score:     ", multi_model_b_scores["train_score"].mean())
print("Validation score:", multi_model_b_scores["test_score"].mean())
print()
print("Previous Model")
print("Train score:     ", multi_model_scores["train_score"].mean())
print("Validation score:", multi_model_scores["test_score"].mean())

In [ ]:
##lets scale our entire X_train_numeric2 dataframe
ss = StandardScaler()
ss.fit(X_train_numeric2)
X_train_numeric2_scaled = ss.transform(X_train_numeric2)

In [ ]:
##using OLS to investigate unexpected result
sm.OLS(y_train, sm.add_constant(X_train_numeric2_scaled)).fit().summary()

**Observations**
- Taking a look at the condition number - it is extremely high, indicating we are violating our multicollinearity   assumption
- This is expected because the 3 sqft features (sqft_living, sqft_above, sqft_basement) we included have correlation between them. (sqft_living is the sum of sqft_above and sqft_basement)
    
- we've also scaled our features so our coefficients are more interpretable. 
- Lets try adding in some of our categorical (non-numerical) features and rerun the model.

### Feature Engineering for Muliple Regression Model 2 

- lets encode our categorical values - waterfront, zipcode, was_renovated, and condition. 
- we'll use the OrdinalEncoder for waterfront and was_renovated - since they have Boolean values. 
- for zipcode and condition we'll create dummy variables using get_dummies from pandas.
- lets also include some columns we created earlier - the ratio and age features.

In [ ]:
##getting values for waterfront features
X_train['waterfront'].value_counts()

In [ ]:
#take boolean column of 'waterfront' and encode
waterfront_train = X_train[["waterfront"]]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_train)
encoder_waterfront.categories_[0]

In [ ]:
#flattening
waterfront_encoded_train = encoder_waterfront.transform(waterfront_train)
waterfront_encoded_train = waterfront_encoded_train.flatten()
waterfront_encoded_train

In [ ]:
#replacing values and checking
X_train["waterfront"] = waterfront_encoded_train
X_train.head()

In [ ]:
#checking newly created values for waterfront feature
X_train['waterfront'].value_counts()

In [ ]:
##getting values for was_renovated feature
X_train['was_renovated'].value_counts()

In [ ]:
#take boolean column of 'was_renovated', encoding, and flattening
was_renovated_train = X_train[["was_renovated"]]
encoder_was_renovated = OrdinalEncoder()
encoder_was_renovated.fit(was_renovated_train)
was_renovated_encoded_train = encoder_was_renovated.transform(was_renovated_train)
was_renovated_encoded_train = was_renovated_encoded_train.flatten()
was_renovated_encoded_train
#replacing in X_train
X_train["was_renovated"] = was_renovated_encoded_train
X_train.head()

In [ ]:
#checking newly created values for was_renovated feature
X_train['was_renovated'].value_counts()

In [ ]:
#getting values for zipcode feature
X_train['zipcode'].value_counts()

In [ ]:
##getting values for condition feature
X_train['condition'].value_counts()

In [ ]:
#lets drop all columns we're not using,
X_train.drop(X_train.columns[[5, 7, 9,14, 15, 16, 17]], axis = 1, inplace = True)
#sanity check
X_train.info()

In [ ]:
#create dummy variables for the "condition" and "zipcode" columns
condition_dummies = pd.get_dummies(X_train['condition'], drop_first=True)
zipcode_dummies = pd.get_dummies(X_train['zipcode'], drop_first=True)

In [ ]:
#merging condition and zipcode dummy datasets to X_train
X_train = pd.concat([X_train, condition_dummies, zipcode_dummies], axis=1)
X_train.head()

In [ ]:
#dropping condition and zipcode original columns
X_train.drop(['condition', 'zipcode'], axis = 1, inplace = True)

In [ ]:
X_train.columns

### Multiple Regression Model 1c
- Now that we've encoded our condition, zipcode, renovation, and waterfront features. Lets throw everything all our     features and analyze our results 
- Lets also scale our X_train to be able to interpret our coefficents with a little more ease
- Still using cross validate to split our training set into three.

In [ ]:
ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)

In [ ]:
multi_model_c = LinearRegression()
##performing 3 separate train/tets splits on our X_train, y_train
splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
#performing regression on all columns in our dataset
multi_model_c_scores = cross_validate(
    estimator=multi_model_c,
    X=X_train_scaled,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Current Model")
print("Train score:     ", multi_model_c_scores["train_score"].mean())
print("Validation score:", multi_model_c_scores["test_score"].mean())
print()
print("Model 1")
print("Train score:     ", multi_model_scores["train_score"].mean())
print("Validation score:", multi_model_scores["test_score"].mean())
print()
print("Model 1B")
print("Train score:     ", multi_model_b_scores["train_score"].mean())
print("Validation score:", multi_model_b_scores["test_score"].mean())

In [ ]:
#Huge jump in our validation score. Let's checkout multicollinearity 
sm.OLS(y_train, sm.add_constant(X_train)).fit().summary()

## Observations
- as expected we have an incredible high condition number indicating several of the features in this model
  are related.  
- let's use RFECV - this will tell us what features are most important to the model without us defining a specific       number of features we want to use. 
- lets also use RFE to get an idea of the top 8 features affecting this model.

In [ ]:
#identifying what features are important to the model.
X_train_for_RFECV = StandardScaler().fit_transform(X_train)

model_for_RFECV = LinearRegression()

selector = RFECV(model_for_RFECV, cv=splitter)
selector.fit(X_train_for_RFECV, y_train)

print("Was the column selected?")
for index, col in enumerate(X_train.columns):
    print(f"{col}: {selector.support_[index]}")

In [ ]:
#identifying the top 8
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select =8)

ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
select.fit(X= X_train_scaled, y=y_train)
select.support_

In [ ]:
#subsetting dataset excluding zipcodes
X_train_heatmap = X_train.filter(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'waterfront', 'sqft_above', 'yr_built',
                          'yr_renovated', 'sqft_basement2', 'was_renovated', 'Distance_to_Water', 'bed_bath_ratio',
                          'bath_bed_ratio', 'sqft_living_to_bedroom_ratio', 'sqft_living_to_bathroom_ratio', 
                          'ratio_sqft_lot_living', 'ratio_sqft_living_lot', 'age_at_sale', 'age_renovation_at_sale',
                          'Fair', 'Good', 'Poor', 'Very Good'], axis=1)

In [ ]:
#visualizing correlations in our dataset excluding zipcodes

mask = np.triu(np.ones_like(X_train_heatmap.corr(), dtype=bool))

plt.figure(figsize=(20,10))

cor = X_train_heatmap.corr().abs()
sns.heatmap(cor, mask=mask, annot=True);

## Observations
- Using both the RFE and RFECV tell us something different. The RFE method says that the top 8 most significant         features are 
    1. age renovation at sale 
    2. sqft living space 
    3. age at sale
    4. year built 
    5. year renovated
    6. sq above
    7. sqft basement
    8. bedrooms
- now we know that the square footage features have multicollinearity among them as seen from the heatmap above, but sqft_living space is the feature that is most correlated with price. So we'll use that in our final model.
- we also can see that waterfront and the zip code feature are very relevant and have no meaningful correlation with other features so let's include them as well. 
- finally we know that bedrooms are given as a top feature - but since we can see that it is correlated with bathrooms and sqft_living lets use one of our bedroom ratios to eliminate multicollinearity and still include the feature in our model.


In [ ]:
X_train.columns

In [ ]:
X_train_final = X_train.filter(['sqft_living', 'waterfront','bedrooms', 'was_renovated', 'bed_bath_ratio',
                                'ratio_sqft_living_lot','sqft_living_to_bathroom_ratio', 'Fair', 'Good', 'Poor',
                                'Very Good', '98002', '98003', '98004', '98005',
                           '98006', '98007', '98008', '98010', '98011', '98014', '98019', '98022',
                           '98023', '98024', '98027', '98028', '98029', '98030', '98031', '98032',
                           '98033', '98034', '98038', '98039', '98040', '98042', '98045', '98052',
                            '98053', '98055', '98056', '98058', '98059', '98065', '98070', '98072',
                           '98074', '98075', '98077', '98092', '98102', '98103', '98105', '98106',
                           '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118',
                           '98119', '98122', '98125', '98126', '98133', '98136', '98144', '98146',
                           '98148', '98155', '98166', '98168', '98177', '98178', '98188', '98198', '98199' ], axis=1)

In [ ]:
multi_model_3 = LinearRegression()

final_model = LinearRegression()
##performing 3 separate train/tets splits on our X_train, y_train
splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
#performing regression on all columns in our dataset
final_model_scores = cross_validate(
    estimator=final_model,
    X=X_train_final,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Final Model")
print("Train score:     ", final_model_scores["train_score"].mean())
print("Validation score:", final_model_scores["test_score"].mean())
print()
print("Model 1C")
print("Train score:     ", multi_model_c_scores["train_score"].mean())
print("Validation score:", multi_model_c_scores["test_score"].mean())
print()
print("Model 1B")
print("Train score:     ", multi_model_b_scores["train_score"].mean())
print("Validation score:", multi_model_b_scores["test_score"].mean())
print()
print("Model 1")
print("Train score:     ", multi_model_scores["train_score"].mean())
print("Validation score:", multi_model_scores["test_score"].mean())

In [ ]:
#lets take a look at multicollinearity using OLS
sm.OLS(y_train, sm.add_constant(X_train_final)).fit().summary()

In [ ]:
##checking model assumptions
p = sns.distplot(final_model.resid);
plt.xlabel("Residual Error", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
p.set_title("Distribution of Residual Error");

resid = final_model.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].hist(resid)
ax[0].set_xlabel("Residual Error")
ax[0].set_ylabel("Count")
ax[0].set_title(label="Histogram of Residual Error");

ax[1].scatter(x=final_model.fittedvalues,y=resid)
ax[1].set_xlabel("Predicted Values")
ax[1].set_ylabel("Residual Error")
ax[1].set_title(label="Test for Homoscedasticity")

plt.style.use('ggplot')
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True);

## Observations
- looks we came up with our best model. 
- Its a combination of the following features:
    1. sqft_living
    2. waterfront
    3. bedrooms
    4. was_renovated
    5. bed_bath_ratio
    6. ratio_sqft_living_lot
    7. sqft_living_to_bathroom_ratio
    8. condition
    9. zipcodes
 Now let's check our assumptions:

In [ ]:
#Checking assumptions

## Test sample
- transforming our sample the way we transformed our training dataset.
- Using our test sample and seeing how accurate our model is**

In [ ]:
#Transforming our sample
#Try scaling coeficients
#Import standard scaler
from sklearn.preprocessing import StandardScaler

#Scale our exogenous variables
ss = StandardScaler()
ss.fit(df_numeric.drop(columns=['price']))
X_scaled = ss.transform(df_numeric.drop(columns=['price']))

#Create dataframe with columns titles
X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df.columns = list(df_numeric.drop(columns=['price']).columns)

In [ ]:
#creating new numeric df from our train split
X_test_numeric =X_test.filter(['bedrooms','bathrooms','sqft_living','sqft_lot','floors',
                                    'sqft_above','yr_built','yr_renovated','lat','long','sqft_living15',
                                    'sqft_lot15','sqft_basement2','Distance_to_Water' ], axis=1)
X_test_numeric.info()


In [ ]:
X_test['waterfront'].value_counts()

In [ ]:
#take boolean column of 'waterfront' and encode
waterfront_test = X_test[["waterfront"]]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_test)
encoder_waterfront.categories_[0]

In [ ]:
#flattening
waterfront_encoded_test = encoder_waterfront.transform(waterfront_test)
waterfront_encoded_test = waterfront_encoded_test.flatten()
waterfront_encoded_test

In [ ]:
#replacing values and checking
X_test["waterfront"] = waterfront_encoded_test
X_test.head()

In [ ]:
#checking newly created values for waterfront feature
X_test['waterfront'].value_counts()

In [ ]:
##getting values for was_renovated feature
X_test['was_renovated'].value_counts()

In [ ]:
#take boolean column of 'was_renovated', encoding, and flattening
was_renovated_test = X_test[["was_renovated"]]
encoder_was_renovated = OrdinalEncoder()
encoder_was_renovated.fit(was_renovated_test)
was_renovated_encoded_test = encoder_was_renovated.transform(was_renovated_test)
was_renovated_encoded_test = was_renovated_encoded_test.flatten()
was_renovated_encoded_test
#replacing in X_test
X_test["was_renovated"] = was_renovated_encoded_test
X_test.head()

In [ ]:
#checking newly created values for was_renovated feature
X_test['was_renovated'].value_counts()

In [ ]:
#getting values for zipcode feature
X_test['zipcode'].value_counts()

In [ ]:
##getting values for condition feature
X_test['condition'].value_counts()

In [ ]:
#lets drop all columns we're not using,
X_test.drop(X_train.columns[[5, 7, 9,14, 15, 16, 17]], axis = 1, inplace = True)
#sanity check
X_test.info()

In [ ]:
#create dummy variables for the "condition" and "zipcode" columns
condition_dummies = pd.get_dummies(X_test['condition'], drop_first=True)
zipcode_dummies = pd.get_dummies(X_test['zipcode'], drop_first=True)

In [ ]:
#merging condition and zipcode dummy datasets to X_train
X_test = pd.concat([X_test, condition_dummies, zipcode_dummies], axis=1)
X_test.head()

In [ ]:
#dropping condition and zipcode original columns
X_test.drop(['condition', 'zipcode'], axis = 1, inplace = True)

In [ ]:
X_test.info()

In [ ]:
ss = StandardScaler()
ss.fit(X_test)
X_test_scaled = ss.transform(X_test)

In [ ]:
X_test_final = X_test.filter(['sqft_living', 'waterfront','bedrooms', 'was_renovated', 'bed_bath_ratio',
                                'ratio_sqft_living_lot','sqft_living_to_bathroom_ratio', 'Fair', 'Good', 'Poor',
                                'Very Good', '98002', '98003', '98004', '98005',
                           '98006', '98007', '98008', '98010', '98011', '98014', '98019', '98022',
                           '98023', '98024', '98027', '98028', '98029', '98030', '98031', '98032',
                           '98033', '98034', '98038', '98039', '98040', '98042', '98045', '98052',
                            '98053', '98055', '98056', '98058', '98059', '98065', '98070', '98072',
                           '98074', '98075', '98077', '98092', '98102', '98103', '98105', '98106',
                           '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118',
                           '98119', '98122', '98125', '98126', '98133', '98136', '98144', '98146',
                           '98148', '98155', '98166', '98168', '98177', '98178', '98188', '98198', '98199' ], axis=1)

In [ ]:
#Scoring our test sample


In [ ]:
##checking model assumptions
p = sns.distplot(final_model.resid);
plt.xlabel("Residual Error", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
p.set_title("Distribution of Residual Error");

resid = final_model.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].hist(resid)
ax[0].set_xlabel("Residual Error")
ax[0].set_ylabel("Count")
ax[0].set_title(label="Histogram of Residual Error");

ax[1].scatter(x=final_model.fittedvalues,y=resid)
ax[1].set_xlabel("Predicted Values")
ax[1].set_ylabel("Residual Error")
ax[1].set_title(label="Test for Homoscedasticity")

plt.style.use('ggplot')
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True);

In [ ]:
#calculating root mean squared error.

## Conclusion